# Getting started with the Python environment and packages

In [1]:
# import the relevant Python packages

import numpy
import nibabel
import nipype
import matplotlib


# The PET Brain Imaging Data Structure

## History

The PET modality is a recent addition to BIDS with its introduction via BEP 009. If you're interested in seeing exactly what and how something gets added to BIDS see the pull request for BEP009 [here](https://github.com/bids-standard/bids-specification/pull/633). The results of that extension proposal can be read [here](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/09-positron-emission-tomography.html#positron-emission-tomography) in the bids standard.

# PET data conversion

## PET image data file formats


Before we start to convert data we need to quickly mention that PET image data files come of the scanner in various different formats, some scanners provide DICOM files (.dcm) and others use proprietary formats for example ECAT format (.v) . In order to facilitate easy testing of data conversion across different PET file formats  the [OpenNeuroPET project](https://openneuropet.github.io/) has compiled a bunch of phantom data from different scanner types and is distributing two examples [here](https://drive.google.com/file/d/10S0H7HAnMmxHNpZLlifR14ykIuiXcBAD/view?usp=sharing) . You can download them for testing purposes either manually or in the terminal by typing 

In [4]:
!pip install gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=e71113090b470abd820e00b5226ff2605077d52d455372ccbdd79048b0d25a80
  Stored in directory: /home/jovyan/.cache/pip/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown


In [6]:
!gdown https://drive.google.com/file/d/10S0H7HAnMmxHNpZLlifR14ykIuiXcBAD/view?usp=sharing --fuzzy

Downloading...
From: https://drive.google.com/uc?id=10S0H7HAnMmxHNpZLlifR14ykIuiXcBAD
To: /home/jovyan/nh2022-curriculum/ganz-petbids/OpenNeuroPET-Demo_raw.zip
100%|███████████████████████████████████████| 53.7M/53.7M [00:00<00:00, 175MB/s]


Now let's look at wat we have downloaded:

In [7]:
!ls .

OpenNeuroPET-Demo_raw.zip  PET_BIDS_tutorial.ipynb  README.md


The correct file is there, so let's unzip the downloaded file:

In [9]:
!unzip OpenNeuroPET-Demo_raw.zip

Archive:  OpenNeuroPET-Demo_raw.zip
   creating: OpenNeuroPET-Demo_raw/
  inflating: __MACOSX/._OpenNeuroPET-Demo_raw  
  inflating: OpenNeuroPET-Demo_raw/.DS_Store  
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._.DS_Store  
   creating: OpenNeuroPET-Demo_raw/sub-SiemensHRRT/
   creating: OpenNeuroPET-Demo_raw/source/
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._source  
   creating: OpenNeuroPET-Demo_raw/code/
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._code  
  inflating: OpenNeuroPET-Demo_raw/README  
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._README  
  inflating: OpenNeuroPET-Demo_raw/.bidsignore  
  inflating: OpenNeuroPET-Demo_raw/dataset_description.json  
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._dataset_description.json  
   creating: OpenNeuroPET-Demo_raw/sub-SiemensBiographNRU/
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._sub-SiemensBiographNRU  
   creating: OpenNeuroPET-Demo_raw/sub-SiemensHRRT/pet/
   creating: OpenNeuroPET-Demo_raw/source/SiemensBiographPETMR

You can now look at the file tree:

In [16]:
!tree OpenNeuroPET-Demo_raw

/bin/bash: tree: command not found


Now you have an example dataset where you have source data (both for ECAT and DICOM PET image format) and the PET BIDS data sets constructed for it.

Also if you have access to another PET image file format, or data from a scanner not tested, please reach out to [OpenNeuroPET project](https://openneuropet.github.io/) in order to add a phantom scan in your format.

## Conversion

The [OpenNeuroPET project](https://openneuropet.github.io/) has tried to develop tools for facilitating easy data conversion for PET. The main tool used for this is [PET2BIDS](https://github.com/openneuropet/PET2BIDS) freely available on the [OpenNeuroPET  GitHub repository](https://github.com/openneuropet) along with other resources like altlases or pipelines. It is available for both Python and MatLab. Eventually, [PET2BIDS](https://github.com/openneuropet/PET2BIDS) will also be wrapped inside other BIDS conversion tools such as [BIDScoin](https://github.com/Donders-Institute/bidscoin) or [ezBIDS](https://brainlife.io/ezbids/), but this is work in progress at the moment. 

Besides using  [PET2BIDS](https://github.com/openneuropet/PET2BIDS) there is always the possibility to manually convert a data set to PET BIDS and an example will be shown below. In any case, for dicom data format, one relies on [dcm2niix](https://www.nitrc.org/plugins/mwiki/index.php/dcm2nii:MainPage).

Below we will show two ways of converting your PET data to BIDS: 1) using [PET2BIDS](https://github.com/openneuropet/PET2BIDS) and 2) manually.

### 1) Conversion of PET data using PET2BIDS

Detailed documentation for PET2BIDS can be found [here](https://pet2bids.readthedocs.io/en/latest/index.html#) or on the [Github repo](https://github.com/openneuropet/PET2BIDS/blob/main/README.md).

#### Get dcm2niix

Download [dcm2niix](https://www.nitrc.org/plugins/mwiki/index.php/dcm2nii:MainPage):

In [ ]:
!conda install -c conda-forge dcm2niix

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::ipywidgets==7.7.0=pyhd8ed1ab_0
  - conda-forge/noarch::jsonschema==4.4.0=pyhd8ed1ab_0
  - conda-forge/noarch::typing_extensions==4.1.1=pyha770c72_0
  - conda-forge/noarch::async_generator==1.10=py_0
  - conda-forge/noarch::certipy==0.1.3=py_0
  - conda-forge/linux-64::greenlet==1.1.2=py37hd23a5d3_2
  - conda-forge/noarch::jupyterlab_widgets==1.1.0=pyhd8ed1ab_0
  - conda-forge/noarch::json5==0.9.5=pyh9f0ad1d_0
  - conda-forge/linux-64::sqlite==3.37.1=h4ff8645_0
  - conda-forge/noarch::beautifulsoup4==4.11.0=pyha770c72_0
  - conda-forge/noarch::nbclassic==0.3.7=pyhd8ed1ab_0
  - conda-forge/noarch::nbformat==5.3.0=pyhd8ed1ab_0
  - conda-forge/noarch::jupyterlab_pygments==0.2.0=pyhd8ed1ab_0
  - conda-forge/noarch::urllib3==1.26.9=pyhd8ed1ab_0
  - conda-forge/linux-64::ruamel.yaml.clib==0.2.6=py37h540881e_1
  - conda-

#### Get the Python package PET2BIDS

In [18]:
pip install pypet2bids 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.2/555.2 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.7/280.7 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 30.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### Convert your first dataset 

Now you already have the converter installed and can go ahead and convert your first dataset! 

In this example, I am converting an image in DICOM format. It should be noted that pypet2bids  contains several different tools and is itself a part of the larger PET library PET2BIDS, the specific tool I will be using for the following DICOM conversion is dcm2niix4pet.

You just need to point dcm2niix4pet to the folder where your data resides, *dcmfolder*, and the folder where you want to output the PET BIDS formatted dataset, *mynewfolder*:

In [21]:
!dcm2niix4pet ./OpenNeuroPET-Demo_raw/source/SiemensBiographPETMR-NRU -d mynewfolder

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/bin/dcm2niix4pet", line 8, in <module>
    sys.exit(main())
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/pypet2bids/dcm2niix4pet.py", line 1022, in main
    silent=cli_args.silent)
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/pypet2bids/dcm2niix4pet.py", line 336, in __init__
    self.check_for_dcm2niix()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/pypet2bids/dcm2niix4pet.py", line 385, in check_for_dcm2niix
    {instructions} and packaged versions can be found at {pkged}""")
Exception: Dcm2niix does not appear to be installed. Installation instructions can be found here 
                   https://github.com/rordenlab/dcm2niix#install and packaged versions can be found at https://github.com/rordenlab/dcm2niix/releases


Note, dcm2niix4pet will do it's best to extract as much information about radiological and blood data from the DICOM files in the dcmfolder. However, dcm2niix4pet can't find information if it isn't there, hence it will often be up to you the user to provide some missing information at the time of conversion. 

Additional information can be provided via the command line with the `--kwargs` argument in the form of key=pair values. For an idea of what this looks like see below:



```bash
dcm2niix4pet /OpenNeuroPET-Demo_raw/source/SiemensBiographPETMR-NRU -d mynewfolder --kwargs TimeZero=ScanStart 
Manufacturer=Siemens 
ManufacturersModelName=Biograph 
InstitutionName="Rigshospitalet, NRU, DK"
BodyPart=Phantom
Units=Bq/mL
TracerName=none
TracerRadionuclide=F18
InjectedRadioactivity=81.24
SpecificRadioactivity=13019.23
ModeOfAdministration=infusion
FrameTimesStart=0
AcquisitionMode="list mode"
ImageDecayCorrected=true
ImageDecayCorrectionTime=0
AttenuationCorrection=MR-corrected
FrameDuration=300
FrameTimesStart=0
```

Now you have a dataset in PET BIDS format. You will probably have gotten some warnings relating to the .json sidecar file. Carefully look at them, since they will help you to catch inconsistencies and missing required fields that you need to add in order for the dataset to pass the BIDS validator as well (see below how that's done). 
You can always edit the .json file, by opening it in a text editor and manually fixing errors. Alternatively, adjust the meta structure you created above to correct the errors.

# PET processing

## PET example data for processing

In the following we will basically follow the tutorial presented in the [PyPetSurfer repository](https://github.com/openneuropet/PET_pipelines/tree/main/pyPetSurfer) and perform a simple data processing of a PET data set. Therefore we need to clone that GitHub repository:


In [ ]:
! git clone https://github.com/openneuropet/PET_pipelines/tree/main/pyPetSurfer .

Then we need to cd to the right directory inside the Github repository:

In [ ]:
 cd pyPetSurfer 

Download this example dataset from OpenNeuro: https://openneuro.org/datasets/ds001421. You can see instructions on how to do this [here](https://openneuro.org/datasets/ds001421/versions/1.2.1/download). In this example, I will use the openneuro cli 

In [ ]:
datalad install https://github.com/OpenNeuroDatasets/ds001421.git

Now let's see if this is there and how it looks like:

In [ ]:
!ls .

And now simply perform your first PET data anlysis, including

In [ ]:
%run example.py